### ISS-LIS_download files_based on user input of time & region of interest_hdffile filtered_v2.0

**Earthdata API is used to access and query NASA data, for details: https://pypi.org/project/earthdata/**

In [1]:
import earthdata
from earthdata import Auth, Store, DataCollections, DataGranules

**To access the cell below one should have to create a .netrc file in home directory, please follow the given steps below:-**

BY CMD IT, COULD BE DONE AS FOLLOWS: 
(1) cd ~ or cd $HOME,
(2) touch .netrc,
(3) echo "machine urs.earthdata.nasa.gov login <uid> password <password>" >> .netrc 
(Note that here <uid> is your user name and <password> is your password used for Earthdata search Login, enter both without the brackets),
(4) chmod 0600 .netrc (so only you can access it)

OR

BY MANUAL PROCESS, IT COULD BE DONE AS FOLLOWS: 
(1) Go to dir
(find your dir: On Windows, this might look like 'C:\Users\<your name>\', on a Mac '/opt/'),
(2) Open notepad, and paste the content in the brackets "machine urs.earthdata.nasa.gov login <uid> password <password>" 
(Note that here <uid> is your user name and <password> is your password used for Earthdata search Login, enter both without the brackets),
(3) Save the notepad as '.netric'

**Finally once the netric file is successfully created in your dir, you would be able to RUN the next cell and it would show the output as "You're now authenticated with NASA Earthdata Login, True"**

In [2]:
### AUTHENTICATION STEP

auth = Auth()
auth.login(strategy="netrc")
# are we authenticated?
print(auth.authenticated)

You're now authenticated with NASA Earthdata Login
True


**Note here we need NASA authentication to access the files directly from GHRC server*

**We can search for collections using a pythonic API client for CMR**

## Please enter the date from 2017-03-01 onwards till date

### Input start time of interest

In [3]:
start_time = input('Enter start time in format: YYYY-MM-DD \n')

Enter start time in format: YYYY-MM-DD 
2021-07-25


### Input end time of interest

In [4]:
end_time = input('Enter end time in format: YYYY-MM-DD \n')

Enter end time in format: YYYY-MM-DD 
2021-07-30


**LET now find Non-Quality Controlled Lightning Imaging Sensor (LIS) on International Space Station (ISS) Science Data V2,  
LIS granules for given dates and access their metadata using earthdata API get() method**

In [5]:
### We build our query, note as ISS_LIS data comes 1file per orbit so we can temporally query the data### 
### direct spatial query not possible in case of ISS_LIS data###
### The short name for collection was found from cell[2]###

from pprint import pprint
Query = DataGranules().short_name('isslis_v2_nqc').temporal(start_time,end_time)

###We get all metadata records###
granules = Query.get()

###Please uncomment to display granules
#print(granules)

In [6]:
### Access bounding rectangle coordinates from each granules (as required to indirectly query ISS_LIS data wrt region of interest)
### Note here these coordinates are ones when stellite is changing its orbit or doing some*
### *maintainance/instrumentation health check

space = []

for i in range(len(granules)):
    space_iss = granules[i]['umm']['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['BoundingRectangles']
    
    ### to remove square brackets from start and end of each string
    space_iss_1 = str(space_iss)[1:-1]
    
    # change datatype from string to dict 
    space_iss_1 = eval(space_iss_1)
    space.append(space_iss_1)
    
###Please uncomment to display bounding rectangle coordinates
#space

In [7]:
###Change the dataframe from dict to dataframe

import pandas as pd
spatial = pd.DataFrame(space)

###Please uncomment to display table of bounding rectangle coordinates 
#spatial

**Now lets try to extract data URLS from the metadata of each datasets of our interest**

In [8]:
data_links = [granule.data_links() for granule in granules]

###Please uncomment to display links
#print(data_links)

**OOPS!! we are not able to access the GHRC DAAC server to directly download the files using get() method from earthdata library so lets take a long cut (atleast for time being)**

**'https' url for GHRC-DAAC server are sorted (from 's3' ie AWS links) and than the bounding rectangle coordinates dataframe is merged with 'https' sorted dataframe**

In [9]:
###covert list to dataframe .. a long cut again :)
import pandas as pd
url = pd.DataFrame(data_links, columns = ['https' , 'sf3'])

###converted dataframe back to list sorting hhtps urls
url_https = url['https']#.values.tolist()
#print(inprem)
type(url_https)

### merge the bounding rectangle coordinates dataframe is merged with 'https' dorted dataframe
result_1 = pd.concat([spatial, url_https], axis=1, join="outer")

result_1.head()

,WestBoundingCoordinate,EastBoundingCoordinate,NorthBoundingCoordinate,SouthBoundingCoordinate,https
0,145.419,119.246,51.735,-51.926,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...
1,145.419,119.246,51.735,-51.926,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...
2,122.839,92.982,51.741,-51.883,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...
3,122.839,92.982,51.741,-51.883,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...
4,96.576,69.409,51.734,-51.926,https://data.ghrc.earthdata.nasa.gov/ghrcw-pro...


In [10]:
### An indirect method to sptially query ISS_LIS data wrt region of interest
### Finally we filter the iss_lis data based on the condition that whenever the bounding coordinates are positive (ie longitude is between 0 to 180degrees) all the satellite orbit are covering canada region

import pandas as pd
result = result_1[(result_1['WestBoundingCoordinate'] > 0) & (result_1['EastBoundingCoordinate'] > 0)]

### sorted coordinates
#result

In [11]:
### Now let's filter list URLs with extension '.hdf' 
### NOTE that here we sort HDF files from list of HDF and NETCDF4 files as generally HDF file size are smaller than NETCDF4 files
### Using List comprehension method (filtering list of strings based on the substring list)

import re
 
### convert dataframe to back to list dtype    
download_https = result["https"].values.tolist()

def Filter(download_https, substr):
    return [str for str in download_https if
             any(sub in str for sub in substr)]
     
substr = ['.hdf']

download_https_hdf = Filter(download_https, substr)

### Please uncomment to display sorted .hdf links
#download_https_hdf

***Finally now, https urls for GHRC-DAAC server are opened using loop and temporarily sorted files are downloaded ;)***

In [13]:
##Works https url opens and data is downloaded to downloads as default location
#import time
import webbrowser

for i in download_https_hdf:
    reponse = webbrowser.open(i)

***THE CODE ENDs HERE***

.

*Management of huge sized data files from NASA is a big challenge and so in this code not only helps to get connected with NASA server using API to get ISS_LIS data files but also further enables to query the data wrt time , bounding rectangle coordinates according to ones research interest and file type (as hdf file size is relatively smaller)**

**ALL THE FILTERING EFFORTS ARE JUST DONE TO OPTIMALLY USE AVAILABLE SPACE IN COMPUTER***

***AS SEEN BELOW, a significant reduction in number of datefiles to be downloaded can be clearly observed ~1/5th wrt the initial total datafiles***

In [14]:
### Total number of data files after inputing time period of interest 
len(granules)

158

In [15]:
### Number of data files would be reduced due to orbital sorting (orbits passing over region of interest:Canada in our case are sorted) 
len(result)

70

In [16]:
### Finally further reduction in the number of datafiles is achieved by sorting hdf files from netcdf files
len(download_https_hdf)

35

.

**Issues yet to be addressed:**
1) Not able to access the files from local DAAC server directly using get() method
(By default the AWS links gets accessed and no file is downloaded)

2) Have used URL open method to download files but all files are saved in downloads: How to change the directory ?

3) It can be seen in the whole code that the datatype is changed from dict to list to pandas dataFrame and this can help to shorten the length of the overall code

**All your inputs to further optimize the pythonic code are would be of great help and welcomed !!**